<a href="https://colab.research.google.com/github/Karthick47v2/FoodMenu/blob/main/frozen_lake_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://gym.openai.com/envs/FrozenLake-v0/

# States 4

# 1 - S - Starting point - Safe
# 2 - F - Forzen lake - Safe 
# 3 - H - Hole - Unsafe
# 4 - G - Goal - Safe

In [50]:
# import 
import gym          # for getting env
import random       # generate random path (number) - exploration
import numpy as np  # q-table
import time         # sleep
from IPython.display import clear_output

In [3]:
# generate frozen-lake-v0 environment
env = gym.make("FrozenLake-v0")

In [4]:
# generate Q-table

# no of rows = no of States
# no of cols = no of Actions

num_actions = env.action_space.n
num_states = env.observation_space.n

# initialize Q-table with all zeros coz, at start we don't have any prior q values

# q_table = np.zeros((num_states, num_actions))

In [49]:
####
## We don't want to train forever, so we need to have some boudnaries
####
#  num of episodes for training
num_episodes = 10000
# num of steps agents takes max per episode
max_steps_per_episode = 100

In [51]:
def train_agent(l_rate=0.1, d_rate=0.99, min_exp_rate=0.01, exp_decay_rate=0.01):
  # initialize Q-table with all zeros coz, at start we don't have any prior q values

  q_table = np.zeros((num_states, num_actions))


  # initialize Q-learning params
  # how much agent adopts from new q-value AKA Learning rate - alpha
  learning_rate = l_rate
  # Make sure agent focus on maximizing rewards for near future AKA discount rate - gamma
  discount_rate = d_rate

  ####
  ## Exploration Vs Exploitation
  ###

  # start with max exploration rate so that agent can learn the env then increase the exploitation rate

  exploration_rate = 1
  max_exploration_rate = 1
  min_exploration_rate = min_exp_rate
  exploration_decay_rate = exp_decay_rate ### 0.001-------------------------------------------------

  # store rewards for each episodes
  rewards_all_episodes = []

  # here is our training starts
  # for all episodes 
  for episode in range(num_episodes):
    # initialize episodic variables
    # rewards for episode
    rewards = 0
    # chekc if episode done
    done = False
    # MOST IMPORTANT ::::: RESET STATE for each episode 
    state = env.reset()

    # now lets see actions for each timestep 
    for step in range(max_steps_per_episode):
      # Explore vs Exploit
      exploration_rate_threshold = random.uniform(0,1)
      action = 0
      # if current exploration rate is less than threshold then explore, else exploit
      if exploration_rate_threshold <= exploration_rate:
        #explore
        action = env.action_space.sample()
      else:
        #exploit
        action = np.argmax(q_table[state, :])
      # print(exploration_rate)
      # now we have chosen the action, lets see the results of action
      new_state, reward, done, info = env.step(action)

      # now update the Q-value on table
      ## FORMULA = q(s,a) = (1 - alpha) * q(s',a') (old q value) + alpha * (reward for current action + discounted max q value for next state-action pair)
      q_table[state,action] = (1 - learning_rate) * q_table[state, action] + learning_rate * (reward + discount_rate * np.max(q_table[new_state,:]))
      # if(reward == 1):
      #   print(reward)
      # change to new state
      state = new_state
      # add reward
      rewards += reward

      # check if episode is done (achieved goal or fell on hole)
      if done:
        # if so, finish episode
        break

    # if not, before going to next episode decay the expolartion rate --- exploration rate will exponentially decay
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)

    # append rewards to list for each episodes
    rewards_all_episodes.append(rewards)

  return rewards_all_episodes, q_table

In [52]:
rewards_all_episodes, q_table = train_agent(l_rate=(0.1), d_rate=0.99, min_exp_rate=0.01, exp_decay_rate=0.001)

# after training finished, lets see results -- avg
rewards_per_1000_episodes = np.split(np.array(rewards_all_episodes), 10)
count = 1000
print("l_rate = ", str(0.1), "d_rate = ", str(0.99), "min_exp_rate = ", str(0.01), "exp_decay_rate = ", str(0.01))

for i in rewards_per_1000_episodes:
  print(count, " : ", str(sum(i) / 1000))
  count += 1000

l_rate =  0.1 d_rate =  0.99 min_exp_rate =  0.01 exp_decay_rate =  0.01
1000  :  0.046
2000  :  0.187
3000  :  0.423
4000  :  0.564
5000  :  0.653
6000  :  0.658
7000  :  0.675
8000  :  0.681
9000  :  0.68
10000  :  0.709


In [55]:
# lets put the agent to test

for episode in range(1,4):
  done = False
  state = env.reset()

  print("EPISODE ", episode)
  time.sleep(1)

  for step in range(max_steps_per_episode):
    clear_output(wait=True)
    # render 
    env.render() 
    time.sleep(0.3)

    # do action
    action = np.argmax(q_table[state,:])
    # result
    new_state, reward, done, info = env.step(action)

    if done:
      clear_output(wait=True)
      env.render()
      
      if reward == 1:
        print("GOAL")
      else:
        print("FAILED")
      time.sleep(3)
      clear_output(wait=True)
      break;
    
    state = new_state

env.close()


  (Down)
SFFF
FHFH
FFFH
HFFG
GOAL
